In [15]:
# Step 0: Imports and global config

import os
import re
import json
import warnings
from datetime import datetime
from typing import Dict, Any, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# Your input folder with ~100 CSVs
INPUT_FOLDER = "cleaned B2B_circ"

# Create a unique run folder so outputs from different runs never clash
RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")
BASE_OUT = os.path.join("ts_outputs", RUN_ID)
os.makedirs(BASE_OUT, exist_ok=True)

# Which numeric fields to model (you can add/remove)
METRIC_COLUMNS = [
    ("Invoice Value", "InvoiceValue"),
    ("Taxable Value", "TaxableValue"),
]

# Minimum number of monthly points required to attempt ARIMA
MIN_POINTS = 9   # since most companies have ~9–13 months
FORECAST_HORIZON = 3  # forecast up to 3 months ahead


# Plot style: simple Matplotlib (no seaborn), larger fonts
plt.rcParams.update({"figure.figsize": (10, 5), "axes.grid": True, "font.size": 11})


In [17]:
# Step 1: Helper functions

def sanitize_name(s: str) -> str:
    """
    Make a filesystem-safe name: keep alphanumerics and underscores.
    """
    s = os.path.splitext(os.path.basename(s))[0]  # drop extension
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^A-Za-z0-9_]", "", s)
    return s or "company"

def parse_date_safe(series: pd.Series) -> pd.Series:
    """
    Try multiple formats for 'Invoice date'. Returns pd.Series with datetime64.
    """
    # First pass: YYYY-MM-DD
    parsed = pd.to_datetime(series, format="%Y-%m-%d", errors="coerce")
    mask = parsed.isna()
    if mask.any():
        # Second pass: DD-MM-YYYY
        parsed.loc[mask] = pd.to_datetime(series[mask], format="%d-%m-%Y", errors="coerce")
    mask = parsed.isna()
    if mask.any():
        # Third pass: DD/MM/YYYY or general with dayfirst
        parsed.loc[mask] = pd.to_datetime(series[mask], dayfirst=True, errors="coerce")
    return parsed

def monthly_agg(df: pd.DataFrame, value_col: str) -> pd.Series:
    """
    Take raw invoice rows and produce a monthly sum series on the 'Invoice date' column.
    """
    tmp = df.copy()
    tmp["Invoice date"] = parse_date_safe(tmp["Invoice date"])
    tmp = tmp.dropna(subset=["Invoice date"])
    # Ensure numeric
    tmp[value_col] = pd.to_numeric(tmp[value_col], errors="coerce")
    tmp = tmp.dropna(subset=[value_col])

    # Set index and resample to monthly sums
    ts = (
        tmp.set_index("Invoice date")[value_col]
           .sort_index()
           .resample("MS")  # Month Start; choose "M" if you prefer Month End
           .sum()
           .astype(float)
    )
    # Fill missing months with 0 (typical for sales-like series)
    ts = ts.asfreq("MS", fill_value=0.0)
    return ts

def ensure_company_outdir(company_safe: str) -> str:
    """
    Create per-company output directory inside this run.
    """
    out_dir = os.path.join(BASE_OUT, company_safe)
    os.makedirs(out_dir, exist_ok=True)
    return out_dir

def save_fig(path: str):
    """
    Tight layout and save current Matplotlib figure to `path`.
    """
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

from statsmodels.tsa.stattools import adfuller, kpss

def run_adf(series):
    series = series.dropna()
    try:
        if series.nunique() <= 1:
            return {"statistic": None, "pvalue": None, "note": "constant_series"}
        result = adfuller(series, autolag="AIC")
        return {
            "statistic": float(result[0]),
            "pvalue": float(result[1]),
            "n_lags": int(result[2]),
            "n_obs": int(result[3]),
            "critical_values": {k: float(v) for k, v in result[4].items()},
        }
    except Exception as e:
        return {"statistic": None, "pvalue": None, "note": str(e)}

def run_kpss(series):
    series = series.dropna()
    try:
        if series.nunique() <= 1:
            return {"statistic": None, "pvalue": None, "note": "constant_series"}
        result = kpss(series, regression="c", nlags="auto")
        return {
            "statistic": float(result[0]),
            "pvalue": float(result[1]),
            "lags": int(result[2]),
            "critical_values": {k: float(v) for k, v in result[3].items()},
        }
    except Exception as e:
        return {"statistic": None, "pvalue": None, "note": str(e)}


def try_fit_arima(series: pd.Series, order: Tuple[int,int,int]):
    """
    Fit ARIMA(order) on the given series.
    Returns fitted model or None if it fails.
    """
    try:
        model = ARIMA(series, order=order, enforce_stationarity=False, enforce_invertibility=False)
        fitted = model.fit()
        return fitted
    except Exception as e:
        return None

def ljung_box_resid(residuals: np.ndarray, lags: int = 12) -> Dict[str, Any]:
    """
    Ljung-Box test on residuals for up to `lags` lags.
    """
    lb = acorr_ljungbox(residuals, lags=[lags], return_df=True)
    # Single lag row (lags column equals provided lags)
    row = lb.iloc[0]
    return {"lags": int(row["lags"]), "lb_stat": float(row["lb_stat"]), "lb_pvalue": float(row["lb_pvalue"])}

def _safe_pacf_lags(n: int) -> int | None:
    # YW methods require nlags < n/2. Return None if too short.
    max_allowed = n // 2 - 1
    if max_allowed < 1:
        return None
    return min(24, max_allowed)



In [18]:
# Step 2: Loop through all companies, create monthly series for each selected metric

all_companies = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(".csv")]
print(f"Found {len(all_companies)} company files.")

catalog_rows = []  # one row per company x metric to track time spans and sizes

for fname in sorted(all_companies):
    company_safe = sanitize_name(fname)
    out_dir = ensure_company_outdir(company_safe)

    df_path = os.path.join(INPUT_FOLDER, fname)
    try:
        df = pd.read_csv(df_path, dtype={
            "GSTIN/UIN of Recipient": "string",
            "Receiver Name": "string",
            "Sender Name": "string",
            "Invoice Number": "string",
            "Invoice date": "string",
            "Invoice Value": "string",
            "Place Of Supply": "string",
            "Reverse Charge": "string",
            "Applicable % of Tax Rate": "string",
            "Invoice Type": "string",
            "E-Commerce GSTIN": "string",
            "Rate": "string",
            "Taxable Value": "string",
            "Cess Amount": "string",
        })
    except Exception as e:
        print(f"Skipping {fname}: read error: {e}")
        continue

    for raw_col, metric_tag in METRIC_COLUMNS:
        if raw_col not in df.columns:
            print(f"Skipping {fname} for metric '{raw_col}': column not found.")
            continue

        ts = monthly_agg(df, raw_col)
        if ts.empty:
            print(f"Skipping {fname} for metric '{raw_col}': no valid dates or values.")
            continue

        # Save the series
        series_csv = os.path.join(
            out_dir, f"{company_safe}__metric-{metric_tag}__freq-MS__series.csv"
        )
        ts.to_csv(series_csv, header=[metric_tag])

        # Quick plot
        plt.figure()
        ts.plot()
        plt.title(f"{company_safe} — Monthly {metric_tag} (sum)")
        plt.xlabel("Month")
        plt.ylabel(metric_tag)
        fig_path = os.path.join(out_dir, f"{company_safe}__metric-{metric_tag}__freq-MS__series.png")
        save_fig(fig_path)

        # Track for catalog
        catalog_rows.append({
            "company_file": fname,
            "company_safe": company_safe,
            "metric": metric_tag,
            "first_month": ts.index.min().strftime("%Y-%m"),
            "last_month": ts.index.max().strftime("%Y-%m"),
            "n_points": int(ts.shape[0]),
            "series_csv": os.path.relpath(series_csv),
            "series_plot": os.path.relpath(fig_path),
        })

# Write a catalog CSV so you can inspect coverage quickly
catalog_df = pd.DataFrame(catalog_rows)
catalog_path = os.path.join(BASE_OUT, "catalog__company_metric_series.csv")
catalog_df.to_csv(catalog_path, index=False)
print(f"\nCatalog written: {catalog_path}")
display(catalog_df.head(10))


Found 100 company files.

Catalog written: ts_outputs\20250830_002548\catalog__company_metric_series.csv


,company_file,company_safe,metric,first_month,last_month,n_points,series_csv,series_plot
0,0L725.csv,0L725,InvoiceValue,2024-01,2025-01,13,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...
1,0L725.csv,0L725,TaxableValue,2024-01,2025-01,13,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...
2,0LVSN.csv,0LVSN,InvoiceValue,2024-01,2024-09,9,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...
3,0LVSN.csv,0LVSN,TaxableValue,2024-01,2024-09,9,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...
4,0Y9DO.csv,0Y9DO,InvoiceValue,2024-01,2024-12,12,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...
5,0Y9DO.csv,0Y9DO,TaxableValue,2024-01,2024-12,12,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...
6,1IBLJ.csv,1IBLJ,InvoiceValue,2024-01,2024-10,10,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...
7,1IBLJ.csv,1IBLJ,TaxableValue,2024-01,2024-10,10,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...
8,1T2TA.csv,1T2TA,InvoiceValue,2024-01,2024-11,11,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...
9,1T2TA.csv,1T2TA,TaxableValue,2024-01,2024-11,11,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...


In [20]:
# Step 3: Stationarity tests (ADF, KPSS) on raw series and 1st difference (d=1)
# - Reads the series created in Step 2 for every company and metric
# - Runs ADF and KPSS on raw and differenced data
# - Saves per-company JSON with full stats, plus raw/diff plots
# - Writes a run-level CSV: stationarity_summary.csv

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Safety: if the catalog from Step 2 is not in memory, load it
if "catalog_df" not in globals():
    catalog_path = os.path.join(BASE_OUT, "catalog__company_metric_series.csv")
    if not os.path.exists(catalog_path):
        raise FileNotFoundError(f"Catalog not found at {catalog_path}. Please run Step 2 first.")
    catalog_df = pd.read_csv(catalog_path)

# Use the same plotting helper if present
def _save_fig_safe(path: str):
    try:
        save_fig(path)  # defined in Step 1
    except NameError:
        plt.tight_layout()
        plt.savefig(path, dpi=150)
        plt.close()

# Minimum points to attempt tests
MIN_FOR_TESTS = max(8, MIN_POINTS)  # respects your new MIN_POINTS = 9

stationarity_rows = []

for _, row in catalog_df.iterrows():
    company_safe = row["company_safe"]
    metric = row["metric"]
    out_dir = os.path.join(BASE_OUT, company_safe)
    series_csv = os.path.join(out_dir, f"{company_safe}__metric-{metric}__freq-MS__series.csv")

    if not os.path.exists(series_csv):
        print(f"Skip {company_safe}-{metric}: series CSV not found.")
        continue

    # Load series
    ts = pd.read_csv(series_csv, index_col=0, parse_dates=True).iloc[:, 0]
    ts = ts.asfreq("MS").astype(float)

    # Plot and save raw series for visual check
    plt.figure()
    ts.plot()
    plt.title(f"{company_safe} - {metric} - RAW series")
    plt.xlabel("Month")
    plt.ylabel(metric)
    _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__raw_series.png"))

    # Prepare differenced series (d=1)
    d1 = ts.diff().dropna()

    # Plot and save differenced series
    plt.figure()
    d1.plot()
    plt.title(f"{company_safe} - {metric} - DIFF1 series")
    plt.xlabel("Month")
    plt.ylabel(metric)
    _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__diff1_series.png"))

    # Run tests only if we have enough points
    adf_raw = run_adf(ts) if len(ts) >= MIN_FOR_TESTS else None
    kpss_raw = run_kpss(ts) if len(ts) >= MIN_FOR_TESTS else None
    adf_d1  = run_adf(d1) if len(d1) >= MIN_FOR_TESTS else None
    kpss_d1 = run_kpss(d1) if len(d1) >= MIN_FOR_TESTS else None

    # Save detailed JSON
    stat_json = {
        "company": company_safe,
        "metric": metric,
        "n_points_raw": int(len(ts)),
        "n_points_diff1": int(len(d1)),
        "adf_raw": adf_raw,
        "kpss_raw": kpss_raw,
        "adf_diff1": adf_d1,
        "kpss_diff1": kpss_d1,
    }
    with open(os.path.join(out_dir, f"{company_safe}__metric-{metric}__stationarity.json"), "w") as f:
        json.dump(stat_json, f, indent=2)

    # Add lightweight row for the run-level CSV
    stationarity_rows.append({
        "company_safe": company_safe,
        "metric": metric,
        "n_points_raw": int(len(ts)),
        "n_points_diff1": int(len(d1)),
        "adf_raw_p": adf_raw["pvalue"] if adf_raw else np.nan,
        "kpss_raw_p": kpss_raw["pvalue"] if kpss_raw else np.nan,
        "adf_d1_p": adf_d1["pvalue"] if adf_d1 else np.nan,
        "kpss_d1_p": kpss_d1["pvalue"] if kpss_d1 else np.nan,
    })

# Save run-level summary CSV
stat_df = pd.DataFrame(stationarity_rows)
stat_csv = os.path.join(BASE_OUT, "stationarity_summary.csv")
stat_df.to_csv(stat_csv, index=False)

print(f"Stationarity summary written: {stat_csv}")
display(stat_df.head(12))


Stationarity summary written: ts_outputs\20250830_002548\stationarity_summary.csv


,company_safe,metric,n_points_raw,n_points_diff1,adf_raw_p,kpss_raw_p,adf_d1_p,kpss_d1_p
0,0L725,InvoiceValue,13,12,0.019825,0.1,7.554036e-05,0.100000
1,0L725,TaxableValue,13,12,0.780178,0.1,5.120823e-04,0.100000
2,0LVSN,InvoiceValue,9,8,0.023894,0.1,NaN,NaN
3,0LVSN,TaxableValue,9,8,0.265923,0.1,NaN,NaN
4,0Y9DO,InvoiceValue,12,11,0.000080,0.1,1.115778e-06,0.041667
5,0Y9DO,TaxableValue,12,11,0.706076,0.1,1.441023e-10,0.100000
6,1IBLJ,InvoiceValue,10,9,0.522587,0.1,5.429450e-02,0.100000
7,1IBLJ,TaxableValue,10,9,0.963790,0.1,3.195368e-08,0.079024
8,1T2TA,InvoiceValue,11,10,0.320156,0.1,3.031671e-01,0.041667
9,1T2TA,TaxableValue,11,10,NaN,NaN,NaN,NaN


In [21]:
# Step 4 (updated): ACF/PACF on d=1, fit ARIMA(2,1,0), robust residual diagnostics, and quiet plotting

import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

# Quiet plotting: prevent inline displays and figure repr spam
plt.ioff()

# Safety: load catalog if not in memory
if "catalog_df" not in globals():
    catalog_path = os.path.join(BASE_OUT, "catalog__company_metric_series.csv")
    if not os.path.exists(catalog_path):
        raise FileNotFoundError(f"Catalog not found at {catalog_path}. Please run Step 2 first.")
    catalog_df = pd.read_csv(catalog_path)

def _save_fig_safe(path: str):
    """Tight layout, save, and close without printing any figure reprs."""
    try:
        save_fig(path)  # from Step 1, if defined
    except NameError:
        plt.tight_layout()
        plt.savefig(path, dpi=150)
        plt.close()

def _try_fit_arima(series: pd.Series, order=(2,1,0)):
    try:
        model = ARIMA(series, order=order, enforce_stationarity=False, enforce_invertibility=False)
        return model.fit()
    except Exception:
        return None

def robust_ljung_box(residuals: pd.Series) -> dict:
    """
    Run Ljung–Box on residuals with safe lag selection.
    Returns a dict with status, lags, lb_stat, lb_pvalue, and optional reason.
    """
    r = residuals.dropna()
    if len(r) < 4:
        return {"lb_status":"skipped", "lb_reason":"too_short", "lags":np.nan, "lb_stat":np.nan, "lb_pvalue":np.nan}
    if np.isclose(r.std(ddof=0), 0.0):
        return {"lb_status":"skipped", "lb_reason":"constant_residuals", "lags":np.nan, "lb_stat":np.nan, "lb_pvalue":np.nan}

    # Candidate lags that respect small samples
    def _cap(n):  # ensure lag < n/2 and >=2
        return max(2, min(12, n // 2 - 1))
    candidates = [_cap(len(r)), 8, 6, 4, 2]
    candidates = [lag for lag in candidates if lag >= 2]

    last_err = None
    for lag in candidates:
        try:
            # Pass an int: this returns rows for lags 1..lag; take the last row
            lb_df = acorr_ljungbox(r, lags=lag, return_df=True)
            row = lb_df.iloc[-1]
            lag_used = int(lb_df.index[-1])  # <- read lag from the index
            return {
                "lb_status": "ok",
                "lags": lag_used,
                "lb_stat": float(row["lb_stat"]),
                "lb_pvalue": float(row["lb_pvalue"]),
            }
        except Exception as e:
            last_err = str(e)

    return {
        "lb_status": "failed",
        "lb_reason": last_err if last_err else "unknown_error",
        "lags": np.nan, "lb_stat": np.nan, "lb_pvalue": np.nan
    }


# We require at least MIN_POINTS points for fitting ARIMA as you configured earlier
required_points = MIN_POINTS if "MIN_POINTS" in globals() else 9

rows = []
for _, r in catalog_df.iterrows():
    company_safe = r["company_safe"]
    metric = r["metric"]
    out_dir = os.path.join(BASE_OUT, company_safe)

    # Load the monthly series
    series_csv = os.path.join(out_dir, f"{company_safe}__metric-{metric}__freq-MS__series.csv")
    if not os.path.exists(series_csv):
        # skip silently to keep output clean
        continue

    ts = pd.read_csv(series_csv, index_col=0, parse_dates=True).iloc[:, 0]
    ts = ts.asfreq("MS").astype(float)

    # Skip if too short to fit
    if len(ts) < required_points:
        rows.append({
            "company_safe": company_safe,
            "metric": metric,
            "n_points": int(len(ts)),
            "fit_status": "skipped_insufficient_points",
            "aic": np.nan,
            "bic": np.nan,
            "lb_status": "skipped",
            "lb_reason": "insufficient_points",
            "lb_lags": np.nan,
            "lb_stat": np.nan,
            "lb_pvalue": np.nan
        })
        continue

    # ACF and PACF of differenced series (d=1)
 # ACF and PACF of differenced series (d=1)
# ACF and PACF of differenced series (d=1)
    d1 = ts.diff().dropna()
    if len(d1) >= 2:
    # ACF is fine up to n-1
        _ = plot_acf(d1, lags=min(24, len(d1) - 1), zero=False)
        plt.title(f"{company_safe} - {metric} - ACF (d=1)")
        _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__acf_d1.png"))

    # PACF: respect nlags < n/2 for YW methods
        pacf_lags = _safe_pacf_lags(len(d1))
        if pacf_lags is not None:
            _ = plot_pacf(d1, lags=pacf_lags, zero=False, method="ywm")
            plt.title(f"{company_safe} - {metric} - PACF (d=1)")
            _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__pacf_d1.png"))
    # else: too short for a safe PACF; skip silently to keep output clean

    # else: too short for a safe PACF; skip silently to keep output clean

    else:
        # not enough for correlograms; skip silently
        pass

    # Fit baseline ARIMA(2,1,0)
    fitted = _try_fit_arima(ts, order=(2,1,0))
    if fitted is None:
        rows.append({
            "company_safe": company_safe,
            "metric": metric,
            "n_points": int(len(ts)),
            "fit_status": "fit_failed",
            "aic": np.nan,
            "bic": np.nan,
            "lb_status": "failed",
            "lb_reason": "fit_failed",
            "lb_lags": np.nan,
            "lb_stat": np.nan,
            "lb_pvalue": np.nan
        })
        continue

    # Save model summary
    with open(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_0__summary.txt"), "w") as f:
        f.write(str(fitted.summary()))

    # Residual diagnostics
    resid = fitted.resid

    # Residual time plot
    plt.figure()
    plt.plot(resid.index, resid.values)
    plt.title(f"{company_safe} - {metric} - Residuals (ARIMA(2,1,0))")
    plt.xlabel("Month")
    plt.ylabel("Residual")
    _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_0__residuals.png"))

    # Residual ACF (only if at least 2 residuals)
    if resid.dropna().shape[0] >= 2:
        _ = plot_acf(resid.dropna(), lags=min(24, resid.dropna().shape[0] - 1), zero=False)
        plt.title(f"{company_safe} - {metric} - Residual ACF (ARIMA(2,1,0))")
        _save_fig_safe(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_0__resid_acf.png"))

    # Ljung–Box with robust fallback and explicit status
    lb_res = robust_ljung_box(resid)

    rows.append({
        "company_safe": company_safe,
        "metric": metric,
        "n_points": int(len(ts)),
        "fit_status": "ok",
        "aic": float(fitted.aic),
        "bic": float(fitted.bic),
        "lb_status": lb_res.get("lb_status"),
        "lb_reason": lb_res.get("lb_reason", ""),
        "lb_lags": lb_res.get("lags"),
        "lb_stat": lb_res.get("lb_stat"),
        "lb_pvalue": lb_res.get("lb_pvalue"),
    })

# Save diagnostics CSV for this step
diag_df = pd.DataFrame(rows)
diag_csv = os.path.join(BASE_OUT, "baseline_arima210_diagnostics.csv")
diag_df.to_csv(diag_csv, index=False)

print(f"Baseline ARIMA diagnostics written: {diag_csv}")
# Show a concise preview
try:
    display(diag_df.head(20))
except Exception:
    print(diag_df.head(20))


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1490: ValueWarning: Matrix is singular. Using pinv.
  warnings.warn("Matrix is singular. Using pinv.", ValueWarning)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1490: ValueWarning: Matrix is singular. Using pinv.
  warnings.warn("Matrix is singular. Using pinv.", ValueWarning)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1490: ValueWarning: Matrix is singular. Using pinv.
  warnings.warn("Matrix is singular. Using pinv.", ValueWarning)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1490: ValueWarning: Matrix is singular. Using pinv.
  warnings.warn("Matrix is singular. Using pinv.", ValueWarning)


Baseline ARIMA diagnostics written: ts_outputs\20250830_002548\baseline_arima210_diagnostics.csv


,company_safe,metric,n_points,fit_status,aic,bic,lb_status,lb_reason,lb_lags,lb_stat,lb_pvalue
0,0L725,InvoiceValue,13,ok,243.022908,243.930663,ok,,5.0,2.555455,0.768121
1,0L725,TaxableValue,13,ok,170.001330,170.909086,ok,,5.0,0.930250,0.967993
2,0LVSN,InvoiceValue,9,ok,146.572715,145.947994,ok,,3.0,0.862839,0.834385
3,0LVSN,TaxableValue,9,ok,108.233021,107.608300,ok,,3.0,3.019136,0.388684
4,0Y9DO,InvoiceValue,12,ok,230.918977,231.510651,ok,,5.0,1.326084,0.932225
5,0Y9DO,TaxableValue,12,ok,141.543766,142.135439,ok,,5.0,1.108210,0.953374
6,1IBLJ,InvoiceValue,10,ok,175.568881,175.406612,ok,,4.0,1.456885,0.834250
7,1IBLJ,TaxableValue,10,ok,127.496357,127.334087,ok,,4.0,8.182354,0.085122
8,1T2TA,InvoiceValue,11,ok,208.342031,208.580355,ok,,4.0,1.888448,0.756267
9,1T2TA,TaxableValue,11,ok,-200.302520,-200.064196,skipped,constant_residuals,NaN,NaN,NaN


In [22]:
# Step 5: Try ARIMA(2,1,1) when residuals from ARIMA(2,1,0) show autocorrelation, compare AIC/BIC, pick winner
# Outputs:
# - Per company: ARIMA(2,1,1) summary and residual plots (only when attempted)
# - Run-level: model_selection_summary.csv recording chosen model per series

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.ioff()  # keep plotting quiet

# ---------- Helpers ----------
def _save_fig_quiet(path: str):
    try:
        save_fig(path)  # if you defined this in Step 1
    except NameError:
        plt.tight_layout()
        plt.savefig(path, dpi=150)
        plt.close()

def robust_ljung_box(residuals: pd.Series) -> dict:
    r = residuals.dropna()
    if len(r) < 4:
        return {"lb_status":"skipped", "lb_reason":"too_short", "lags":np.nan, "lb_stat":np.nan, "lb_pvalue":np.nan}
    if np.isclose(r.std(ddof=0), 0.0):
        return {"lb_status":"skipped", "lb_reason":"constant_residuals", "lags":np.nan, "lb_stat":np.nan, "lb_pvalue":np.nan}

    # try decreasing safe lags
    for lag in [min(12, len(r)-1), min(8, len(r)-1), min(6, len(r)-1), 4, 2]:
        if lag >= 2:
            try:
                lb_df = acorr_ljungbox(r, lags=lag, return_df=True)
                row = lb_df.iloc[-1]
                return {"lb_status":"ok", "lags":int(lag), "lb_stat":float(row["lb_stat"]), "lb_pvalue":float(row["lb_pvalue"])}
            except Exception as e:
                last_err = str(e)
    return {"lb_status":"failed", "lb_reason": last_err if 'last_err' in locals() else "unknown_error",
            "lags":np.nan, "lb_stat":np.nan, "lb_pvalue":np.nan}

def try_fit(series: pd.Series, order):
    try:
        m = ARIMA(series, order=order, enforce_stationarity=False, enforce_invertibility=False)
        return m.fit()
    except Exception:
        return None

# ---------- Load inputs ----------
# Catalog may be needed to locate series files
if "catalog_df" not in globals():
    catalog_path = os.path.join(BASE_OUT, "catalog__company_metric_series.csv")
    if not os.path.exists(catalog_path):
        raise FileNotFoundError(f"Missing {catalog_path}. Run Step 2 first.")
    catalog_df = pd.read_csv(catalog_path)

# Baseline diagnostics from Step 4 (prefer BASE_OUT; fall back to /mnt/data if needed)
diag_csv_candidates = [
    os.path.join(BASE_OUT, "baseline_arima210_diagnostics.csv"),
    "/mnt/data/baseline_arima210_diagnostics.csv",
]
for _p in diag_csv_candidates:
    if os.path.exists(_p):
        diag_csv_path = _p
        break
else:
    raise FileNotFoundError("baseline_arima210_diagnostics.csv not found. Run Step 4 first.")

diag_df = pd.read_csv(diag_csv_path)

required_points = MIN_POINTS if "MIN_POINTS" in globals() else 9

# ---------- Iterate and optionally fit ARIMA(2,1,1) ----------
selection_rows = []

for _, r in diag_df.iterrows():
    company_safe = r["company_safe"]
    metric = r["metric"]
    fit_status = r.get("fit_status", "ok")
    aic_210 = r.get("aic", np.nan)
    bic_210 = r.get("bic", np.nan)
    lb_status = r.get("lb_status", "skipped")
    lb_p_210 = r.get("lb_pvalue", np.nan)

    out_dir = os.path.join(BASE_OUT, company_safe)
    series_csv = os.path.join(out_dir, f"{company_safe}__metric-{metric}__freq-MS__series.csv")
    if not os.path.exists(series_csv) or fit_status != "ok":
        # keep a row so every series is represented
        selection_rows.append({
            "company_safe": company_safe, "metric": metric,
            "attempted_211": False, "chosen_model": "ARIMA(2,1,0)",
            "aic_210": aic_210, "bic_210": bic_210,
            "aic_211": np.nan, "bic_211": np.nan,
            "lb_pvalue_210": lb_p_210, "lb_pvalue_211": np.nan
        })
        continue

    ts = pd.read_csv(series_csv, index_col=0, parse_dates=True).iloc[:, 0].asfreq("MS").astype(float)
    if len(ts) < required_points:
        selection_rows.append({
            "company_safe": company_safe, "metric": metric,
            "attempted_211": False, "chosen_model": "ARIMA(2,1,0)",
            "aic_210": aic_210, "bic_210": bic_210,
            "aic_211": np.nan, "bic_211": np.nan,
            "lb_pvalue_210": lb_p_210, "lb_pvalue_211": np.nan
        })
        continue

    # Decide if we should try ARIMA(2,1,1)
    try_211 = (lb_status == "ok") and pd.notna(lb_p_210) and (lb_p_210 < 0.05)

    if not try_211:
        selection_rows.append({
            "company_safe": company_safe, "metric": metric,
            "attempted_211": False, "chosen_model": "ARIMA(2,1,0)",
            "aic_210": aic_210, "bic_210": bic_210,
            "aic_211": np.nan, "bic_211": np.nan,
            "lb_pvalue_210": lb_p_210, "lb_pvalue_211": np.nan
        })
        continue

    # Fit ARIMA(2,1,1)
    m211 = try_fit(ts, (2, 1, 1))
    if m211 is None:
        selection_rows.append({
            "company_safe": company_safe, "metric": metric,
            "attempted_211": True, "chosen_model": "ARIMA(2,1,0)",
            "aic_210": aic_210, "bic_210": bic_210,
            "aic_211": np.nan, "bic_211": np.nan,
            "lb_pvalue_210": lb_p_210, "lb_pvalue_211": np.nan
        })
        continue

    # Save ARIMA(2,1,1) summary
    with open(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_1__summary.txt"), "w") as f:
        f.write(str(m211.summary()))

    # Residual diagnostics for 2,1,1
    resid_211 = m211.resid

    # Residual time plot
    plt.figure()
    plt.plot(resid_211.index, resid_211.values)
    plt.title(f"{company_safe} - {metric} - Residuals (ARIMA(2,1,1))")
    plt.xlabel("Month"); plt.ylabel("Residual")
    _save_fig_quiet(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_1__residuals.png"))

    # Residual ACF
    if resid_211.dropna().shape[0] >= 2:
        _ = plot_acf(resid_211.dropna(), lags=min(24, resid_211.dropna().shape[0]-1), zero=False)
        plt.title(f"{company_safe} - {metric} - Residual ACF (ARIMA(2,1,1))")
        _save_fig_quiet(os.path.join(out_dir, f"{company_safe}__metric-{metric}__ARIMA_2_1_1__resid_acf.png"))

    # Ljung–Box for 2,1,1
    lb_211 = robust_ljung_box(resid_211)

    # Compare models
    aic_211 = float(m211.aic)
    bic_211 = float(m211.bic)

    # Primary: pick lower AIC. If tie within 1e-6, use BIC.
    if aic_211 + 1e-6 < aic_210:
        chosen = "ARIMA(2,1,1)"
    elif abs(aic_211 - aic_210) <= 1e-6 and bic_211 < bic_210:
        chosen = "ARIMA(2,1,1)"
    else:
        chosen = "ARIMA(2,1,0)"

    selection_rows.append({
        "company_safe": company_safe,
        "metric": metric,
        "attempted_211": True,
        "chosen_model": chosen,
        "aic_210": aic_210, "bic_210": bic_210,
        "aic_211": aic_211, "bic_211": bic_211,
        "lb_pvalue_210": lb_p_210,
        "lb_pvalue_211": lb_211.get("lb_pvalue")
    })

# Save run-level selection summary
selection_df = pd.DataFrame(selection_rows)
selection_csv = os.path.join(BASE_OUT, "model_selection_summary.csv")
selection_df.to_csv(selection_csv, index=False)

print(f"Model selection summary written: {selection_csv}")
try:
    display(selection_df.head(20))
except Exception:
    print(selection_df.head(20))


Model selection summary written: ts_outputs\20250830_002548\model_selection_summary.csv


,company_safe,metric,attempted_211,chosen_model,aic_210,bic_210,aic_211,bic_211,lb_pvalue_210,lb_pvalue_211
0,0L725,InvoiceValue,False,"ARIMA(2,1,0)",243.022908,243.930663,NaN,NaN,0.768121,NaN
1,0L725,TaxableValue,False,"ARIMA(2,1,0)",170.001330,170.909086,NaN,NaN,0.967993,NaN
2,0LVSN,InvoiceValue,False,"ARIMA(2,1,0)",146.572715,145.947994,NaN,NaN,0.834385,NaN
3,0LVSN,TaxableValue,False,"ARIMA(2,1,0)",108.233021,107.608300,NaN,NaN,0.388684,NaN
4,0Y9DO,InvoiceValue,False,"ARIMA(2,1,0)",230.918977,231.510651,NaN,NaN,0.932225,NaN
5,0Y9DO,TaxableValue,False,"ARIMA(2,1,0)",141.543766,142.135439,NaN,NaN,0.953374,NaN
6,1IBLJ,InvoiceValue,False,"ARIMA(2,1,0)",175.568881,175.406612,NaN,NaN,0.834250,NaN
7,1IBLJ,TaxableValue,False,"ARIMA(2,1,0)",127.496357,127.334087,NaN,NaN,0.085122,NaN
8,1T2TA,InvoiceValue,False,"ARIMA(2,1,0)",208.342031,208.580355,NaN,NaN,0.756267,NaN
9,1T2TA,TaxableValue,False,"ARIMA(2,1,0)",-200.302520,-200.064196,NaN,NaN,NaN,NaN


In [23]:
# Step 6: Fit chosen model and produce 3-month forecasts with 95% CIs
# - Reads chosen model from model_selection_summary.csv (Step 5)
# - Falls back to ARIMA(2,1,0) when needed
# - Saves per-company forecast CSV + plot
# - Writes run-level forecast_index.csv

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

plt.ioff()  # keep plotting quiet

# ---------- Helpers ----------
def _save_fig_quiet(path: str):
    try:
        save_fig(path)  # if defined in Step 1
    except NameError:
        plt.tight_layout()
        plt.savefig(path, dpi=150)
        plt.close()

def try_fit(series: pd.Series, order):
    try:
        m = ARIMA(series, order=order, enforce_stationarity=False, enforce_invertibility=False)
        return m.fit()
    except Exception:
        return None

def model_to_order(name: str):
    name = str(name).strip().upper()
    if name == "ARIMA(2,1,1)":
        return (2,1,1)
    # default
    return (2,1,0)

# ---------- Load inputs ----------
# Catalog helps locate per-series CSVs
if "catalog_df" not in globals():
    catalog_path = os.path.join(BASE_OUT, "catalog__company_metric_series.csv")
    if not os.path.exists(catalog_path):
        raise FileNotFoundError(f"Missing {catalog_path}. Run Step 2 first.")
    catalog_df = pd.read_csv(catalog_path)

# Model selection from Step 5 (search common locations)
ms_candidates = [
    os.path.join(BASE_OUT, "model_selection_summary.csv"),
    "/mnt/data/model_selection_summary.csv",
]
for _p in ms_candidates:
    if os.path.exists(_p):
        selection_path = _p
        break
else:
    raise FileNotFoundError("model_selection_summary.csv not found. Run Step 5 first.")

selection_df = pd.read_csv(selection_path)

# Baseline diagnostics from Step 4 to see which series fitted successfully
diag_candidates = [
    os.path.join(BASE_OUT, "baseline_arima210_diagnostics.csv"),
    "/mnt/data/baseline_arima210_diagnostics.csv",
]
diag_path = None
for _p in diag_candidates:
    if os.path.exists(_p):
        diag_path = _p
        break
if diag_path:
    diag_df = pd.read_csv(diag_path)
else:
    diag_df = None

required_points = MIN_POINTS if "MIN_POINTS" in globals() else 9
h = FORECAST_HORIZON if "FORECAST_HORIZON" in globals() else 3

# ---------- Forecast all series ----------
rows = []

# If diagnostics are available, join to only forecast where Step 4 said fit_status == ok
if diag_df is not None and {"company_safe","metric","fit_status"}.issubset(diag_df.columns):
    eligible = pd.merge(selection_df, diag_df[["company_safe","metric","fit_status"]], on=["company_safe","metric"], how="left")
else:
    eligible = selection_df.copy()
    eligible["fit_status"] = "ok"  # best effort

for _, r in eligible.iterrows():
    company_safe = r["company_safe"]
    metric = r["metric"]
    chosen = r.get("chosen_model", "ARIMA(2,1,0)")
    fit_status_prev = r.get("fit_status", "ok")

    out_dir = os.path.join(BASE_OUT, company_safe)
    series_csv = os.path.join(out_dir, f"{company_safe}__metric-{metric}__freq-MS__series.csv")

    # Default row to record status
    base_rec = {
        "company_safe": company_safe,
        "metric": metric,
        "chosen_model": chosen,
        "horizon": h,
        "forecast_csv": "",
        "forecast_plot": "",
        "status": "",
        "reason": ""
    }

    if not os.path.exists(series_csv):
        base_rec.update({"status":"skipped", "reason":"series_csv_missing"})
        rows.append(base_rec)
        continue

    ts = pd.read_csv(series_csv, index_col=0, parse_dates=True).iloc[:, 0].asfreq("MS").astype(float)

    if len(ts) < required_points:
        base_rec.update({"status":"skipped", "reason":"insufficient_points"})
        rows.append(base_rec)
        continue

    # If Step 4 fit failed for this series, we still try to fit now using chosen model
    order = model_to_order(chosen)
    model = try_fit(ts, order)
    if model is None:
        base_rec.update({"status":"fit_failed", "reason":"chosen_model_fit_failed"})
        rows.append(base_rec)
        continue

    # Forecast next h months
    fc = model.get_forecast(steps=h)
    fc_mean = fc.predicted_mean

    # 95% CI
    ci = fc.conf_int(alpha=0.05)
    if isinstance(ci, pd.DataFrame):
        lower = ci.iloc[:, 0]
        upper = ci.iloc[:, 1]
    else:
        lower = pd.Series(ci[:, 0], index=fc_mean.index)
        upper = pd.Series(ci[:, 1], index=fc_mean.index)

    fc_df = pd.DataFrame({
        "forecast": fc_mean,
        "lower_95": lower,
        "upper_95": upper,
    })

    # Save forecast CSV
    model_tag = "ARIMA_2_1_1" if order == (2,1,1) else "ARIMA_2_1_0"
    fc_csv = os.path.join(out_dir, f"{company_safe}__metric-{metric}__{model_tag}__forecast_{h}m.csv")
    fc_df.to_csv(fc_csv, index_label="Month")

    # Plot history + forecast
    plt.figure(figsize=(11, 5))
    plt.plot(ts.index, ts.values, label="History")
    plt.plot(fc_df.index, fc_df["forecast"], label="Forecast")
    plt.fill_between(fc_df.index, fc_df["lower_95"], fc_df["upper_95"], alpha=0.25, label="95% CI")
    plt.title(f"{company_safe} - {metric} - {model_tag} - {h}-month forecast")
    plt.xlabel("Month")
    plt.ylabel(metric)
    plt.legend()
    fc_png = os.path.join(out_dir, f"{company_safe}__metric-{metric}__{model_tag}__forecast_{h}m.png")
    _save_fig_quiet(fc_png)

    base_rec.update({
        "forecast_csv": os.path.relpath(fc_csv),
        "forecast_plot": os.path.relpath(fc_png),
        "status": "ok",
        "reason": ""
    })
    rows.append(base_rec)

# Save run-level index of forecasts
forecast_index = pd.DataFrame(rows)
fi_csv = os.path.join(BASE_OUT, "forecast_index.csv")
forecast_index.to_csv(fi_csv, index=False)

print(f"Forecast index written: {fi_csv}")
try:
    display(forecast_index.head(20))
except Exception:
    print(forecast_index.head(20))


Forecast index written: ts_outputs\20250830_002548\forecast_index.csv


,company_safe,metric,chosen_model,horizon,forecast_csv,forecast_plot,status,reason
0,0L725,InvoiceValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,ok,
1,0L725,TaxableValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,ok,
2,0LVSN,InvoiceValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ok,
3,0LVSN,TaxableValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ok,
4,0Y9DO,InvoiceValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ok,
5,0Y9DO,TaxableValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ok,
6,1IBLJ,InvoiceValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ok,
7,1IBLJ,TaxableValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ok,
8,1T2TA,InvoiceValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ok,
9,1T2TA,TaxableValue,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ok,


In [24]:
# Step 7: Master report joining everything into one CSV

import os
import pandas as pd

base = BASE_OUT  # from earlier steps

paths = {
    "catalog": os.path.join(base, "catalog__company_metric_series.csv"),
    "stationarity": os.path.join(base, "stationarity_summary.csv"),
    "diag210": os.path.join(base, "baseline_arima210_diagnostics.csv"),
    "selection": os.path.join(base, "model_selection_summary.csv"),
    "forecast_index": os.path.join(base, "forecast_index.csv"),
}

# Fallbacks if needed (mounted /mnt/data copies)
for key, p in list(paths.items()):
    if not os.path.exists(p):
        alt = f"/mnt/data/{os.path.basename(p)}"
        if os.path.exists(alt):
            paths[key] = alt

dfs = {k: pd.read_csv(v) for k, v in paths.items()}

master = (
    dfs["catalog"]
      .merge(dfs["stationarity"], on=["company_safe","metric"], how="left", suffixes=("","_stat"))
      .merge(dfs["diag210"], on=["company_safe","metric"], how="left", suffixes=("","_diag"))
      .merge(dfs["selection"], on=["company_safe","metric"], how="left", suffixes=("","_sel"))
      .merge(dfs["forecast_index"], on=["company_safe","metric"], how="left", suffixes=("","_fc"))
)

out_csv = os.path.join(base, "MASTER_TIME_SERIES_REPORT.csv")
master.to_csv(out_csv, index=False)
print(f"Master report written: {out_csv}")

# Quick peek
try:
    display(master.head(20))
except Exception:
    print(master.head(20))


Master report written: ts_outputs\20250830_002548\MASTER_TIME_SERIES_REPORT.csv


,company_file,company_safe,metric,first_month,last_month,n_points,series_csv,series_plot,n_points_raw,n_points_diff1,...,aic_211,bic_211,lb_pvalue_210,lb_pvalue_211,chosen_model_fc,horizon,forecast_csv,forecast_plot,status,reason
0,0L725.csv,0L725,InvoiceValue,2024-01,2025-01,13,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,13,12,...,NaN,NaN,0.768121,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,ok,NaN
1,0L725.csv,0L725,TaxableValue,2024-01,2025-01,13,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,13,12,...,NaN,NaN,0.967993,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0L725\0L725__metric...,ts_outputs\20250830_002548\0L725\0L725__metric...,ok,NaN
2,0LVSN.csv,0LVSN,InvoiceValue,2024-01,2024-09,9,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,9,8,...,NaN,NaN,0.834385,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ok,NaN
3,0LVSN.csv,0LVSN,TaxableValue,2024-01,2024-09,9,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,9,8,...,NaN,NaN,0.388684,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ts_outputs\20250830_002548\0LVSN\0LVSN__metric...,ok,NaN
4,0Y9DO.csv,0Y9DO,InvoiceValue,2024-01,2024-12,12,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,12,11,...,NaN,NaN,0.932225,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ok,NaN
5,0Y9DO.csv,0Y9DO,TaxableValue,2024-01,2024-12,12,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,12,11,...,NaN,NaN,0.953374,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ts_outputs\20250830_002548\0Y9DO\0Y9DO__metric...,ok,NaN
6,1IBLJ.csv,1IBLJ,InvoiceValue,2024-01,2024-10,10,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,10,9,...,NaN,NaN,0.834250,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ok,NaN
7,1IBLJ.csv,1IBLJ,TaxableValue,2024-01,2024-10,10,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,10,9,...,NaN,NaN,0.085122,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ts_outputs\20250830_002548\1IBLJ\1IBLJ__metric...,ok,NaN
8,1T2TA.csv,1T2TA,InvoiceValue,2024-01,2024-11,11,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,11,10,...,NaN,NaN,0.756267,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ok,NaN
9,1T2TA.csv,1T2TA,TaxableValue,2024-01,2024-11,11,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,11,10,...,NaN,NaN,NaN,NaN,"ARIMA(2,1,0)",3,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ts_outputs\20250830_002548\1T2TA\1T2TA__metric...,ok,NaN


In [ ]:
# # Step X: Build anomaly features from time-series outputs (add after MASTER report step)
# # Output:
# #   BASE_OUT/features/company_month_anomaly_features.parquet
# #   (optional) BASE_OUT/features/company_month_anomaly_features.csv
# #
# # What this creates (per company × month × metric row):
# #   - value_last: latest month value
# #   - z_last: z-score of latest value vs recent past (leakage-safe)
# #   - spike/drop flags (level-based), surge/crash flags (MoM-change-based)
# #   - lb_flag: residual autocorrelation flag from ARIMA diagnostics
# #   - ratio_tx_iv & low_ratio_flag: taxable vs invoice ratio (joined across metrics)
# #   - ts_available, month, model_tag

# import os
# import numpy as np
# import pandas as pd
# from pathlib import Path

# # ---------- Checks & paths ----------
# try:
#     BASE_OUT  # noqa
# except NameError:
#     raise RuntimeError("BASE_OUT is not defined. Please set BASE_OUT to your time-series run folder before running this cell.")

# master_path = os.path.join(BASE_OUT, "MASTER_TIME_SERIES_REPORT.csv")
# if not os.path.exists(master_path):
#     # Fallback to mounted /mnt/data if you ran earlier steps there
#     alt = "/mnt/data/MASTER_TIME_SERIES_REPORT.csv"
#     master_path = alt if os.path.exists(alt) else master_path
# if not os.path.exists(master_path):
#     raise FileNotFoundError(f"MASTER file not found at: {master_path}. Run the MASTER step first.")

# master = pd.read_csv(master_path)

# # Helper to pick a column from multiple possible names (due to merges)
# def pick_col(df, candidates):
#     for c in candidates:
#         if c in df.columns:
#             return c
#     # soft match on substrings if exact not found
#     for c in df.columns:
#         for k in candidates:
#             if k.lower() in c.lower():
#                 return c
#     return None

# col_company = pick_col(master, ["company_safe"])
# col_metric  = pick_col(master, ["metric"])
# col_lb_p    = pick_col(master, ["lb_pvalue", "lb_pvalue_210", "lb_pvalue_diag"])
# col_model   = pick_col(master, ["chosen_model", "chosen_model_sel"])

# if not col_company or not col_metric:
#     raise KeyError("MASTER file must contain company and metric columns (e.g., 'company_safe', 'metric').")

# # ---------- Series loader ----------
# def load_series(company_safe: str, metric: str):
#     """Load monthly series CSV created earlier. Returns a float Series at MS frequency, or None."""
#     series_csv = os.path.join(BASE_OUT, company_safe, f"{company_safe}__metric-{metric}__freq-MS__series.csv")
#     if not os.path.exists(series_csv):
#         return None
#     s = pd.read_csv(series_csv, index_col=0, parse_dates=True).iloc[:, 0]
#     # Ensure monthly start frequency and float dtype
#     s = s.asfreq("MS").astype(float)
#     # Fill missing months (no activity) as 0 for accounting-type series
#     s = s.fillna(0.0)
#     return s

# # ---------- Feature builder ----------
# rows = []
# for _, row in master.iterrows():
#     comp   = str(row[col_company])
#     metric = str(row[col_metric])

#     ts = load_series(comp, metric)
#     # Build a default record (keeps coverage even if series missing)
#     rec = {
#         "company_safe": comp,
#         "metric": metric,
#         "ts_available": 0,
#         "month": pd.NaT,
#         "value_last": np.nan,
#         "z_last": np.nan,
#         "mom_pct": np.nan,
#         "spike_flag": 0,
#         "drop_flag": 0,
#         "mom_surge_flag": 0,
#         "mom_crash_flag": 0,
#         "lb_flag": 0,
#         "model_tag": str(row.get(col_model, "ARIMA(2,1,0)")) if col_model else "ARIMA(2,1,0)",
#     }

#     # Residual autocorrelation flag from MASTER diagnostics (if present)
#     try:
#         lbp = float(row[col_lb_p]) if (col_lb_p and pd.notna(row[col_lb_p])) else np.nan
#         rec["lb_flag"] = int(pd.notna(lbp) and (lbp < 0.05))
#     except Exception:
#         rec["lb_flag"] = 0

#     if ts is None or ts.dropna().empty:
#         rows.append(rec)
#         continue

#     rec["ts_available"] = 1
#     rec["month"] = ts.index.max()
#     rec["value_last"] = float(ts.iloc[-1])

#     # Leakage-safe baseline: compute stats on PAST only (exclude last point)
#     if len(ts) >= 2:
#         past = ts.iloc[:-1]
#         # rolling window length: at least 3, up to 6, but <= len(past)
#         w = int(min(6, max(3, len(past))))
#         base = past.tail(w)
#         mu = base.mean()
#         sd = base.std(ddof=1)
#         if pd.notna(sd) and sd > 0:
#             rec["z_last"] = float((rec["value_last"] - mu) / sd)

#         prev = float(ts.iloc[-2])
#         rec["mom_pct"] = float((rec["value_last"] - prev) / (abs(prev) + 1e-6)) if prev == prev else np.nan  # safe div

#         # Flags (thresholds are sensible defaults; tune later on validation)
#         # Level-based surprise
#         if pd.notna(rec["z_last"]):
#             rec["spike_flag"] = int(rec["z_last"] >= 2.5)
#             rec["drop_flag"]  = int(rec["z_last"] <= -2.5)

#         # MoM-change surprise z on past MoM distribution (also leakage-safe)
#         past_mom = past.pct_change().replace([np.inf, -np.inf], np.nan).dropna()
#         if len(past_mom) >= 2:
#             m_mu = past_mom.tail(w).mean()
#             m_sd = past_mom.tail(w).std(ddof=1)
#             if pd.notna(m_sd) and m_sd > 0 and pd.notna(rec["mom_pct"]):
#                 mom_z = float((rec["mom_pct"] - m_mu) / m_sd)
#                 rec["mom_surge_flag"] = int(mom_z >= 2.5)
#                 rec["mom_crash_flag"] = int(mom_z <= -2.5)

#     rows.append(rec)

# feat = pd.DataFrame(rows)

# # ---------- Cross-metric join (Invoice vs Taxable ratios) ----------
# iv = feat[feat["metric"] == "InvoiceValue"][["company_safe", "month", "value_last"]].rename(columns={"value_last": "iv_last"})
# tx = feat[feat["metric"] == "TaxableValue"][["company_safe", "month", "value_last"]].rename(columns={"value_last": "tx_last"})
# mt = pd.merge(iv, tx, on=["company_safe", "month"], how="outer")

# mt["ratio_tx_iv"] = mt["tx_last"] / (mt["iv_last"] + 1e-6)
# # Threshold can be tuned; 0.30 is a common first-pass red flag
# mt["low_ratio_flag"] = (mt["ratio_tx_iv"] < 0.30).astype("int8")

# # Attach ratio flags back to BOTH metric rows (so either row carries the context)
# feat = feat.merge(mt[["company_safe", "month", "ratio_tx_iv", "low_ratio_flag"]],
#                   on=["company_safe", "month"], how="left")

# # ---------- Save outputs ----------
# out_dir = os.path.join(BASE_OUT, "features")
# Path(out_dir).mkdir(parents=True, exist_ok=True)

# parquet_path = os.path.join(out_dir, "company_month_anomaly_features.parquet")
# feat.to_parquet(parquet_path, index=False)

# # Optional CSV for eyeballing (comment out if not needed)
# csv_path = os.path.join(out_dir, "company_month_anomaly_features.csv")
# feat.to_csv(csv_path, index=False)

# print(f"Anomaly features written:\n- {parquet_path}\n- {csv_path}")
# print(f"Rows: {len(feat):,} | Columns: {len(feat.columns)}")
# feat.head(10)


Anomaly features written:
- ts_outputs\20250830_002548\features\company_month_anomaly_features.parquet
- ts_outputs\20250830_002548\features\company_month_anomaly_features.csv
Rows: 200 | Columns: 15


,company_safe,metric,ts_available,month,value_last,z_last,mom_pct,spike_flag,drop_flag,mom_surge_flag,mom_crash_flag,lb_flag,model_tag,ratio_tx_iv,low_ratio_flag
0,0L725,InvoiceValue,1,2025-01-01,56911.06,-1.942691,-6.348682e-01,0,0,0,1,0,"ARIMA(2,1,0)",0.013899,1
1,0L725,TaxableValue,1,2025-01-01,790.98,-0.717241,-5.363432e-01,0,0,0,0,0,"ARIMA(2,1,0)",0.013899,1
2,0LVSN,InvoiceValue,1,2024-09-01,89950.38,-1.196645,-4.046071e-01,0,0,0,0,0,"ARIMA(2,1,0)",0.026542,1
3,0LVSN,TaxableValue,1,2024-09-01,2387.44,-1.000105,-4.570323e-01,0,0,0,0,0,"ARIMA(2,1,0)",0.026542,1
4,0Y9DO,InvoiceValue,1,2024-12-01,42507.16,-2.447129,-7.480547e-01,0,0,0,0,0,"ARIMA(2,1,0)",0.000630,1
5,0Y9DO,TaxableValue,1,2024-12-01,26.76,-1.264701,2.676000e+07,0,0,1,0,0,"ARIMA(2,1,0)",0.000630,1
6,1IBLJ,InvoiceValue,1,2024-10-01,4590.02,-2.609395,-9.297945e-01,0,1,0,0,0,"ARIMA(2,1,0)",0.000000,1
7,1IBLJ,TaxableValue,1,2024-10-01,0.00,-1.173443,-1.000000e+00,0,0,0,0,0,"ARIMA(2,1,0)",0.000000,1
8,1T2TA,InvoiceValue,1,2024-11-01,52539.64,-2.839284,-5.543529e-01,0,1,0,0,0,"ARIMA(2,1,0)",0.000000,1
9,1T2TA,TaxableValue,1,2024-11-01,0.00,NaN,0.000000e+00,0,0,0,0,0,"ARIMA(2,1,0)",0.000000,1
